In [114]:
import os
from uuid import uuid4
import time
import requests
from ftplib import FTP
import boto3
from botocore.exceptions import NoCredentialsError
from boto3.s3.transfer import TransferConfig

# 호스트 파일
HOST = "52.79.99.166"
# HOST = "127.0.0.1"

#### 실험 내용

파일 크기를 바꾸면서, 각 방식 별로 총 3번씩 시도

| 파일크기 | 기존안 | 기획안 | 대안 |
| ---    | ---  | ---  | --- |
| 10MB   | 1.16 s ± 7.89 ms | 1.18 s ± 7.8 ms | **1.01 s ± 6.33 ms** |
| 64MB | 7.29 s ± 441 ms | 7.08 s ± 94.1 ms|  **6.03 s ± 121 ms** |
| 128MB  | 14.4 s ± 312 ms | 17.1 s ± 234 ms | **11.8 s ± 63.8 ms** |

#### 네트워크 환경 : 
* 클라이언트 : 자택
* 서버 : AWS ap-northeast-2 ec2 

### 테스트용 파일 구성하기

In [122]:
file_path = "./bigdata.txt"

size_in_mb = 64

def create_random_file(file_path, size_in_mb):
    size_in_bytes = size_in_mb * 1024 * 1024  # MB를 바이트로 변환
    
    with open(file_path, 'wb') as f:
        f.write(os.urandom(size_in_bytes))

create_random_file(file_path, size_in_mb)

### (1) 장고 서버를 통해 파일 업로드하기

In [124]:
%%timeit -r 3 -n 1
s = time.time()

with open(file_path, "rb") as f:
    file = {'file': f}
    # POST 요청으로 파일 업로드 실행
    response = requests.post(
        f'http://{HOST}/upload/', 
        files={'file': f},
        timeout=3600
    )

print(f"업로드 완료: {time.time() - s:.3f}")

업로드 완료: 2.477
업로드 완료: 2.353
업로드 완료: 2.399
2.41 s ± 51.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### (2) FTP 서버를 통해 파일 업로드하기

In [119]:
%%timeit -r 3 -n 1

s = time.time()

ftp = FTP(HOST)
ftp.login(user="user", passwd="password")
ftp.set_pasv(True)
key = str(uuid4())
with open(file_path, 'rb') as file:
    ftp.storbinary(f'STOR {key}', file)
ftp.quit()
print(f"업로드 완료{key}: {time.time() - s:.3f}")

업로드 완료67f76362-2ebf-45e4-9007-0feb51dea4b4: 0.342
업로드 완료b12953d1-6484-4f7f-91ce-cacf1d55e6e4: 0.302
업로드 완료70857c18-16ce-4b9b-b326-e84e3c0100c4: 0.304
316 ms ± 18.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### (3) boto3을 통한 직접 올리기

In [120]:
s3_client = boto3.client('s3',
                         endpoint_url=f"http://{HOST}:9000",
                         aws_access_key_id="minio-access-key",
                         aws_secret_access_key="minio-secret-key",
                         region_name='us-east-1',
                         use_ssl=False)

def upload_file_to_minio(file_path, bucket_name):
    global s3_client

    key = str(uuid4())        
    s = time.time()
    s3_client.upload_file(file_path, bucket_name, key)
    print(f"업로드 완료({key}): {time.time() - s:.3f}")

In [121]:
%%timeit -r 3 -n 1
upload_file_to_minio(
    file_path, 
    "direct-bucket"
)

업로드 완료(60cda495-9a91-42c5-9220-73c79d940607): 0.390
업로드 완료(0fdfcdb1-8c05-4183-94e1-14e27dec0806): 0.343
업로드 완료(15f1309e-77de-4b4a-a2f8-e78401d7dca4): 0.355
363 ms ± 19.7 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
